# Data Prep

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.getcwd()

In [ ]:
# if necessary, change to drone_steering directory.
os.chdir('../..')
os.getcwd()

In [ ]:
from app_local.module import DataEnsembler, GestureTransformer

In [ ]:
de = DataEnsembler(ms_per_frame=120)

data_directory = 'data/gesture/'
de.investigate_available_datafiles( data_dir = data_directory)

de.combined_data_files_df

In [ ]:
de.load_data()
de.assemble_data(tolerance_range = 600, max_error = 700)
de.display_information()

In [ ]:
feature_names = list(de.LabelGenerators[0].feature_names)
feat_dict = {feature:feature_names.index(feature) for feature in feature_names}
feat_dict

In [ ]:
gt = GestureTransformer(feature_names)
P = gt.transform(de.X)

# Plotting Part

In [ ]:
j = 2
print(de.combined_data_files_df.iloc[j])
print("")

lgen = de.LabelGenerators[j]
y = lgen.y
n = len(lgen.y)
print("n:",n)
print("")

gt = GestureTransformer(byrow=True, feature_names=feature_names)
X = gt.transform(lgen.X)

de.LabelGenerators[j].label_df.join(de.LabelGenerators[j].label_info)

Display the movement endpoints (in Milliseconds) of all non-zero labeled gestures:

In [ ]:
lgen.sequence_end_time[lgen.y > 0]

Display the indices of movement ends of all non-zero labeled gestures:

In [ ]:
np.where(lgen.y > 0)

In [ ]:
key = 'leftWrist'
kx = key + '_x'
ky = key + '_y'

# get the label which is used in the current file
lab = y[y>0][0]

this_set = True
if not this_set: #use all the data
    movementData = P[np.isclose(de.y,lab),:,:]
else:
    movementData = X[np.isclose(y,lab),:,:]
    
    
print(movementData.shape)
movementData = movementData[:,:,[feat_dict[kx],feat_dict[ky]]]
avgMovement = movementData.mean(axis = 0)
avgMovement

In [ ]:
import matplotlib.pyplot as plt

axes = plt.gca()
axes.set_xlim([-2,2])
axes.set_ylim([-2,2])
plt.plot(avgMovement[:,0],avgMovement[:,1],marker = "o")
plt.title("avgMovement with label " + str(int(lab)))
plt.show()
plt.close()

In [ ]:
def squared_error(a,b,c,d):
    return round(((a-b)**2 + (c-d)**2).sum(),2)

### Plot I: plot each sample

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
#print(feat_dict)


key = 'leftWrist'
kx = key + '_x'
ky = key + '_y'


for i in range(0,n):
    #specify ax range
    axes = plt.gca()
    axes.set_xlim([-2,2])
    axes.set_ylim([-2,2])
    
    plt.plot(X[i,:,feat_dict[kx]],X[i,:,feat_dict[ky]], marker = "o")

    plt.title("Index: " + str(i) + "  " + "Time " + str(round(lgen.sequence_end_time[i]/1000,2)) + "  " +\
              "Label: " + str(int(y[i]))) 

    plt.show()
    plt.close()


### Plot II: plot with rolling window

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter


key = 'leftWrist'
kx = key + '_x'
ky = key + '_y'

# get data
d = lgen.labeled_data
s = lgen.X.shape[1]
print("#data points:",len(d),'\t',"sequence length:",s)

dims = d[feature_names].shape

D = d[feature_names].values.reshape(1,dims[0],dims[1])
D = gt.transform(D).reshape(dims[0],dims[1])

Dx = D[:,feat_dict[kx]]
Dy = D[:,feat_dict[ky]]

In [ ]:
for i in range(190,220):
#for i in range(0,dims[0]):
    #specify ax range
    axes = plt.gca()
    axes.set_xlim([-2,2])
    axes.set_ylim([-2,2])
    
    tail, head = [5,5]
    
    # movement data
    l1, u1 = i - s, i
    if l1 < 0:
        l1 = 0
    
    # tail -- what happened before
    l2, u2 = i - s - tail, i-s
    if l2 < 0:
        l2 = 0
    if u2 < 0:
        u2 = 0
        
    # head -- what happened after
    l3, u3 = i, i + head
    if u3 > dims[0]:
        u3 = dims[0]
    
    # the frames in the movement
    plt.plot(
        Dx[l1:u1],
        Dy[l1:u1], 
        marker = "o"
    )
    # the future
    plt.plot(
        Dx[l3:u3],
        Dy[l3:u3],
        "r+"
    )
    # the past
    plt.plot(
        Dx[l2:u2+1],
        Dy[l2:u2+1],
        linestyle = "--",
        color = "grey"
    )
    # the current frame
    plt.plot(
        Dx[u1-1],
        Dy[u1-1],
        color = "black",
        marker = "x"
    )

    plt.title("Index: " + str(i) + "  " +\
              "Time " + str(round(d.time[i]/1000,2)) + "  " +\
              "Label: " + str(int(d.label[i]))
             ) 

    plt.show()
    plt.close()